A padding oracle attack is a cryptographic attack that exploits the way some cryptographic algorithms handle padding. It's a powerful attack that allows an attacker to decrypt encrypted data without knowing the encryption key. Here's an in-depth explanation, assuming a background in number theory and programming but not specifically in cryptography.

### Padding in Cryptography

Many cryptographic algorithms operate on fixed-size blocks of data (e.g., 16 bytes for AES). When the plaintext is not a multiple of the block size, it must be padded to fill the last block. A common padding scheme is PKCS7, where the padding consists of a sequence of bytes, each of which is the number of padding bytes added.

For example, if 3 bytes of padding are needed, the padding would be `03 03 03`.

### What is a Padding Oracle?

A padding oracle is a system that reveals whether a given ciphertext has correct or incorrect padding after decryption. This can be an explicit error message, a timing difference, or any other observable behavior that allows an attacker to distinguish between correct and incorrect padding.

### The Padding Oracle Attack

The padding oracle attack exploits the information leaked by the padding oracle to decrypt ciphertexts.

#### 1. Understanding CBC Mode

In Cipher Block Chaining (CBC) mode, each block of ciphertext is decrypted and then XORed with the previous block of ciphertext to obtain the plaintext. The first block is XORed with an Initialization Vector (IV).

The decryption of block \(i\) in CBC mode can be expressed as:

\[
\text{{Plaintext}}[i] = \text{{Decryption}}(\text{{Ciphertext}}[i]) \oplus \text{{Ciphertext}}[i-1]
\]

#### 2. Exploiting the Padding Oracle

The attacker can manipulate the IV or a previous block of ciphertext and observe the padding oracle's response to deduce information about the plaintext.

#### 3. Decrypting One Block

To decrypt one block of ciphertext, the attacker proceeds byte by byte:

- **Choose a Byte to Attack**: Start with the last byte and work backward.
- **Manipulate the IV/Ciphertext**: For each possible value (0 to 255) of the targeted byte, modify the corresponding byte in the IV or previous ciphertext block and submit it to the padding oracle.
- **Observe the Oracle's Response**: If the padding is correct, the attacker knows that the manipulated byte in the IV/ciphertext, when XORed with the corresponding byte in the decrypted block, produces the correct padding value.
- **Deduce the Plaintext Byte**: Calculate the plaintext byte using the known value that produced correct padding and the original value of the manipulated byte in the IV/ciphertext.
- **Repeat for Each Byte**: Repeat this process for each byte in the block, adjusting the manipulated bytes to produce the expected padding for each step.

#### 4. Decrypting Multiple Blocks

The attack can be extended to multiple blocks by manipulating the previous block of ciphertext instead of the IV. The process is the same as for the first block.

### Why It's Powerful

The padding oracle attack is powerful because it only requires the ability to send arbitrary ciphertexts to the server and observe whether the padding is correct or incorrect. It doesn't require knowledge of the encryption key or any other secret information.

### Mitigations

- **Avoid Leaking Padding Information**: Don't provide any information about padding errors. Treat padding errors the same as any other decryption failure.
- **Use Authenticated Encryption**: Authenticated encryption modes like AES-GCM provide integrity checks that prevent the manipulation of ciphertexts.

### Conclusion

The padding oracle attack is an elegant and powerful attack that exploits subtle information leaks in cryptographic implementations. It illustrates the principle that secure encryption involves not just strong algorithms but also careful handling of all aspects of the encryption and decryption process.

In [17]:
import socket
from tqdm.auto import tqdm

def padding_oracle_attack(initial_guess):
    # Initial IV and ciphertext (all zeros)
    iv = bytes(16)
    ciphertext = bytes(16)
    
    # Placeholder for the decrypted block
    decrypted_block = [0] * 16
    
    # Iterate through each byte in the block
    for byte_position in tqdm(range(15, -1, -1)):
        # Padding value for the current byte position
        padding_value = 16 - byte_position
        
        # Iterate through possible byte values (0 to 255)
        for guess in tqdm(range(256)):
            guess = (initial_guess[byte_position] + guess) % 256
            # Craft a new IV based on the current guess and known decrypted bytes
            crafted_iv = bytearray(iv)
            for i in range(byte_position + 1, 16):
                crafted_iv[i] ^= decrypted_block[i] ^ padding_value
            crafted_iv[byte_position] ^= guess
            
            # Send the crafted IV and ciphertext to the server
            if connection.send(crafted_iv, ciphertext):
                # If padding is correct, store the decrypted byte value
                decrypted_byte = guess ^ padding_value
                decrypted_block[byte_position] = decrypted_byte
                break
    
    # Convert the decrypted block to bytes and return
    return bytes(decrypted_block)


class ServerConnection:
    def __init__(self):
        self.HOST = "aes.sstf.site"
        self.PORT = 1337
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect((self.HOST, self.PORT))

    def send(self, iv, ciphertext):
        # Receive and wait for the IV prompt
        self.socket.recv(1024)
        self.socket.sendall(iv.hex().encode() + b'\n')

        # Receive and wait for the ciphertext prompt
        self.socket.recv(1024)
        self.socket.sendall(ciphertext.hex().encode() + b'\n')

        # Receive the response
        response = self.socket.recv(1024).decode()

        # Analyze the response to determine if the padding is correct
        # You may need to adjust this condition based on the server's response
        if "Try again." in response:
            print(f"response: {response}")
            return False
        elif "Wrong CipherText." in response:
            return response
        else:
            print(f"response: {response}")
            raise ValueError()
            # return response
        
    def close(self):
        self.socket.close()
        
connection = ServerConnection()

In [15]:
output = """30%
76/256 [00:58<02:15, 1.33it/s]
54%
139/256 [01:46<01:28, 1.33it/s]
3%
8/256 [00:06<03:08, 1.31it/s]
72%
185/256 [02:20<00:54, 1.31it/s]
7%
17/256 [00:13<03:02, 1.31it/s]
56%
144/256 [01:49<01:24, 1.33it/s]
58%
148/256 [01:53<01:24, 1.28it/s]
19%
49/256 [00:38<02:37, 1.31it/s]
28%
72/256 [00:55<02:20, 1.31it/s]
48%
124/256 [01:34<01:40, 1.31it/s]
79%
203/256 [02:34<00:40, 1.31it/s]
76%
195/256 [02:28<00:46, 1.31it/s]
45%
116/256 [01:29<01:48, 1.29it/s]
13%
34/256 [00:26<02:49, 1.31it/s]
29%
73/256 [00:56<02:19, 1.31it/s]
36%
92/256 [01:10<02:03, 1.33it/s]"""

decrypted_block_ints = []
for line in output.split('\n'):
    if '256' in line:
        byte = line.split('/256')[0]
        decrypted_block_ints.append(int(byte))
        
decrypted_block_ints

[76, 139, 8, 185, 17, 144, 148, 49, 72, 124, 203, 195, 116, 34, 73, 92]

In [12]:
decrypted_block = padding_oracle_attack(decrypted_block_ints[::-1])

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

...

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

In [14]:
decrypted_block

b'LF,y\xcf\xc0vA9\x93\x96\x14\xbd\x0b\x89M'

In [20]:
connection = ServerConnection()

# Desired plaintext, padded to 16 bytes
desired_plaintext = b"CBC Magic!" + b'\x06' * 6
print(len(desired_plaintext), desired_plaintext)

# Calculate the IV
iv = bytes([a ^ b for a, b in zip(decrypted_block, desired_plaintext)])

# Ciphertext is zero bytes
ciphertext = bytes(16)

print(len(iv), iv)
print(len(ciphertext), ciphertext)
# Send the IV and ciphertext to the server
print()
response = connection.send(iv, ciphertext)

# Print the response
print(response)

16 b'CBC Magic!\x06\x06\x06\x06\x06\x06'
16 b'\x0f\x04oY\x82\xa1\x11(Z\xb2\x90\x12\xbb\r\x8fK'
16 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

response: SCTF{CBC_p4dd1n9_0racle_477ack_5tArts_h3re}



ValueError: 